In [1]:
# 필요한 라이브러리 임포트
import json
from datetime import datetime
from typing import Literal
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

In [2]:
# 환경 설정
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [3]:
# 에이전트 목록
members = ["search", "manse", "general_qa"]
options_for_next = ["FINISH"] + members

print(f"현재 시간: {now}")
print(f"사용 가능한 에이전트: {members}")
print(f"분기 옵션: {options_for_next}")


현재 시간: 2025-07-22 15:21:26
사용 가능한 에이전트: ['search', 'manse', 'general_qa']
분기 옵션: ['FINISH', 'search', 'manse', 'general_qa']


In [ ]:
# RouteResponse 모델 정의
class RouteResponse(BaseModel):
    next: Literal["FINISH", "search", "manse", "general_qa"]


In [ ]:
# Supervisor 프롬프트 정의
supervisor_system_prompt = (
    f"오늘 날짜는 {{now}}입니다.\n"
    "당신은 다음과 같은 전문 에이전트들을 조율하는 Supervisor입니다: {members}.\n"
    "입력(사용자 질문)에 따라 가장 적합한 에이전트로 한 번만 분기하세요. "
    "이미 답변을 생성한 에이전트/질문으로 반복 분기하지 마세요. 무한루프에 빠지지 않도록 주의하세요.\n\n"
    "이전에 보냈던 분기로 연속해서 보내지 마세요."

    "각 에이전트의 역할은 다음과 같습니다:\n"
    "- search: 용어/개념/정의/이론/분류/설명/공식/자료/논문/출처 등, 예를 들어 '불속성이 뭐야?', '오행 각각의 의미', '정관 정의', '십신 종류', '사주에서 겁재는?', '오행 설명' 등 정보성 질문에서만 사용하세요. 운세 풀이/미래/해석/내 운세/금전운/재물운 등 해석이나 미래 흐름을 묻는 질문은 절대 search로 보내지 마세요.\n"
    "사용자가 '겁재', '정관', '오행', '십신', '명리', '사주 용어', '이론', '공식 정의', '개념', '분류', '근거', '출처', '자료', '논문', '문서', 'DB', 'pdf', '설명', '분석' 등 '개념', '정의', '이론', '용어 설명'을 묻거나 자료/출처/공식 설명을 요구할 때만 사용하세요. search는 반드시 용어/개념/정의/분류/이론/공식/자료/논문 등 정보성 질문에서만 사용해야 하며, 운세 풀이(사주해석)는 절대 하지 않습니다.\n"
    "사주에 대해서 자세한 설명이 필요하면 retriever를 사용해 답합니다.\n"
    "십신분석의 개념, 사주개념, 또는 사주 오행의 개념적 질문이 들어오면, web를 사용해 답합니다..\n"
    "search 노드는 답변만 생성하며, 일반적인 고민/일상/잡담/추천/선택/음식 등에는 절대 사용하지 않습니다.\n"

    "- manse: 생년월일/시간 등 사주풀이, 운세 해석, 상세 분석 담당. "
    "생년월일/시간/성별/운세 관련 정보가 있을 때만 사용."
    "사주의 개념적질문, 일상 메뉴, 잡담, 선택, 고민 등은 manse로 보내지 않습니다..n"    
    "'겁재가 뭐야?', '오행 설명해줘', '십신의 의미' 등 용어/개념/정의/설명/이론 질문은 절대 manse에서 처리하지 마세요.\n"

    "- general_qa: 일반 상식, 생활 정보, 건강, 공부, 영어, 주식, 투자, 프로그래밍, 고민 상담, **일상 잡담**, **일반적인 질문**이나 '대화형 질문', **음식**, 선택, 추천, 오늘 할 일, 무엇을 고를지, 등등은 모두 general_qa 담당**\n"
    "일반/일상/잡담/선택형 질문에 대해 이미 general_qa가 답변을 생성했다면,"
    "만약 general_qa에서 답변이 이미 생성된 경우에는 state에 birth_info, saju_result가 있어도 다시 manse로 보내지 말고 반드시 FINISH(종료)로 넘기세요."

    "예시)\n"
    "Q: '겁재가 뭐야?' '오행이 뭔가요?' '정관 정의 알려줘.', '불속성이 뭐야?' '수기운 설명해줘' → search\n"
    "Q: '1995년생 남자, 3월 5일 오후 3시' '내 대운에 대해서 알려줘', '내 금전운에 대해서 자세히 알려줘' → manse\n"
    "Q: '오늘 뭐 먹지?', '기분전환 메뉴 추천', '공부법 알려줘',', '영어회화 공부법',  → general_qa\n"
)

In [6]:
supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", supervisor_system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "위 대화를 참고하여, 다음 중 누가 다음 행동을 해야 하는지 선택하세요: {options}"),
])

# Supervisor 에이전트 함수
def supervisor_agent(messages):
    supervisor_chain = (
        supervisor_prompt.partial(options=str(options_for_next), members=", ".join(members), now=now)
        | llm.with_structured_output(RouteResponse)
    )
    
    # State 형태로 메시지 구성
    state = {"messages": messages}
    route_response = supervisor_chain.invoke(state)
    return route_response.next

In [8]:
# 테스트 함수
def test_supervisor_routing(question, chat_history=None):
    """
    주어진 질문에 대해 supervisor가 어떤 에이전트로 분기하는지 테스트합니다.
    
    Args:
        question (str): 테스트할 질문
        chat_history (list): 이전 대화 기록 (선택사항)
    """
    if chat_history is None:
        chat_history = []
    
    # 현재 질문을 메시지에 추가
    messages = chat_history + [HumanMessage(content=question)]
    
    print(f"질문: {question}")
    print(f"대화 기록 길이: {len(chat_history)}")
    
    try:
        # Supervisor 분기 결정
        routing_decision = supervisor_agent(messages)
        
        print(f"\n분기 결정: {routing_decision}")
        
        # 분기 결과 해석
        if routing_decision == "search":
            print("→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기")
        elif routing_decision == "manse":
            print("→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기")
        elif routing_decision == "general_qa":
            print("→ 일반적인 질문/일상 대화로 판단되어 'general_qa' 에이전트로 분기")
        elif routing_decision == "FINISH":
            print("→ 대화 종료로 판단")
        
        return routing_decision
        
    except Exception as e:
        print(f"오류 발생: {e}")
        return None


In [27]:
def simulate_multiple_questions(questions: list):
    """
    여러 질문을 차례로 넣고, 대화 맥락을 유지하면서 supervisor 분기 결과를 출력합니다.
    """
    chat_history = []
    step = 1

    for question in questions:
        print(f"\n[질문 {step}] {question}")

        # 현재까지의 대화 맥락 + 이번 질문 메시지 구성
        messages = chat_history + [HumanMessage(content=question)]

        # supervisor 분기 결정
        next_agent = supervisor_agent(messages)

        print(f"[질문 {step}] 분기 결과: {next_agent}")

        # AI 응답 시뮬레이션 (간단히 예시)
        if next_agent == "manse":
            ai_response = "사주 풀이 결과입니다."
        elif next_agent == "search":
            ai_response = "개념 설명 결과입니다."
        elif next_agent == "general_qa":
            ai_response = "일반 질문 답변입니다."
        elif next_agent == "FINISH":
            print("[대화 종료] FINISH 반환, 종료합니다.")
            break
        else:
            ai_response = "알 수 없는 에이전트 응답입니다."

        # 대화 맥락에 질문과 응답 추가
        chat_history.append(HumanMessage(content=question))
        chat_history.append(AIMessage(content=ai_response))

        step += 1

# 예시 질문 리스트
questions = [
    "1995년 3월 28일 12시 출생 남자 운세 알려줘",  # manse 예상
    "겁재가 뭐야?",  # search 예상
    "오늘 점심 뭐 먹지?",  # general_qa 예상
    "내 대운과 건강운 알려줘",  # manse 예상
    "내 지지가 뭐야?"  # manse 예상
]

simulate_multiple_questions(questions)



[질문 1] 1995년 3월 28일 12시 출생 남자 운세 알려줘
[질문 1] 분기 결과: manse

[질문 2] 겁재가 뭐야?
[질문 2] 분기 결과: search

[질문 3] 오늘 점심 뭐 먹지?
[질문 3] 분기 결과: general_qa

[질문 4] 내 대운과 건강운 알려줘
[질문 4] 분기 결과: manse

[질문 5] 내 지지가 뭐야?
[질문 5] 분기 결과: search


In [24]:
chat_history = []

# 1. 구체적 사주 정보 질문 (manse 분기 예상)
question1 = "1995년 3월 28일 12시 출생 남자에 대해 알려줘"
print("질문1:")
answer1 = test_supervisor_routing(question1, chat_history)

# AI가 답변했다고 가정하고 AIMessage 추가 (대화 맥락 유지용)
chat_history.append(HumanMessage(content=question1))
chat_history.append(AIMessage(content=answer1))

# 2. 같은 세션에서 사주 개념 질문 (search 분기 예상)
question2 = "겁재가 뭐야?"
print("질문2:")
answer2 = test_supervisor_routing(question2, chat_history)
chat_history.append(HumanMessage(content=question2))
chat_history.append(AIMessage(content=answer2))

print("\n" + "="*50 + "\n")

question3 = "내일 뭐먹을까??"
print("질문3:")
answer3 = test_supervisor_routing(question3, chat_history)
chat_history.append(HumanMessage(content=question3))
chat_history.append(AIMessage(content=answer3))


질문1:
질문: 1995년 3월 28일 12시 출생 남자에 대해 알려줘
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
질문2:
질문: 겁재가 뭐야?
대화 기록 길이: 2

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기


질문3:
질문: 내일 뭐먹을까??
대화 기록 길이: 4

분기 결정: general_qa
→ 일반적인 질문/일상 대화로 판단되어 'general_qa' 에이전트로 분기


In [25]:
chat_history

[HumanMessage(content='1995년 3월 28일 12시 출생 남자에 대해 알려줘', additional_kwargs={}, response_metadata={}),
 AIMessage(content='manse', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='겁재가 뭐야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='search', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내일 뭐먹을까??', additional_kwargs={}, response_metadata={}),
 AIMessage(content='general_qa', additional_kwargs={}, response_metadata={})]

In [11]:
question = "겁재가 뭐야?"
test_supervisor_routing(question)

질문: 겁재가 뭐야?
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기


'search'

In [14]:

# Search 에이전트 테스트 케이스
search_test_cases = [
    "겁재가 뭐야?",
    "오행이 뭔가요?",
    "정관 정의 알려줘.",
    "불속성이 뭐야?",
    "수기운 설명해줘",
    "십신의 종류에 대해 알려줘",
    "사주에서 정인은 무엇인가요?",
    "오행의 상생상극 관계를 설명해주세요",
    "명리학의 기본 개념은?"
]

print("=== Search 에이전트 테스트 ===")
for i, question in enumerate(search_test_cases, 1):
    print(f"\n{i}. ")
    test_supervisor_routing(question)
    print("-" * 50)


=== Search 에이전트 테스트 ===

1. 
질문: 겁재가 뭐야?
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------------------------------

2. 
질문: 오행이 뭔가요?
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------------------------------

3. 
질문: 정관 정의 알려줘.
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------------------------------

4. 
질문: 불속성이 뭐야?
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------------------------------

5. 
질문: 수기운 설명해줘
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------------------------------

6. 
질문: 십신의 종류에 대해 알려줘
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------------------------------

7. 
질문: 사주에서 정인은 무엇인가요?
대화 기록 길이: 0

분기 결정: search
→ 용어/개념/정의/이론 관련 질문으로 판단되어 'search' 에이전트로 분기
--------------------------

In [17]:

# Manse 에이전트 테스트 케이스
manse_test_cases = [
    "1995년생 남자, 3월 5일 오후 3시",
    "내 대운에 대해서 알려줘",
    "내 금전운에 대해서 자세히 알려줘",
    "1996년 12월 13일 남자, 10시 30분 출생 운세봐줘.",
    "대운과 세운, 조심해야 할것들 알려줘",
    "금전운알려줘",
    "내 애정운은 어떨까요?",
    "건강운에 대해 분석해주세요",
    "직업운을 봐주세요",
    "겁재가뭐야?"
]


print("=== Manse 에이전트 테스트 ===")
for i, question in enumerate(manse_test_cases, 1):
    print(f"\n{i}. ")
    test_supervisor_routing(question)
    print("-" * 50)

=== Manse 에이전트 테스트 ===

1. 
질문: 1995년생 남자, 3월 5일 오후 3시
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
--------------------------------------------------

2. 
질문: 내 대운에 대해서 알려줘
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
--------------------------------------------------

3. 
질문: 내 금전운에 대해서 자세히 알려줘
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
--------------------------------------------------

4. 
질문: 1996년 12월 13일 남자, 10시 30분 출생 운세봐줘.
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
--------------------------------------------------

5. 
질문: 대운과 세운, 조심해야 할것들 알려줘
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
--------------------------------------------------

6. 
질문: 금전운알려줘
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이전트로 분기
--------------------------------------------------

7. 
질문: 내 애정운은 어떨까요?
대화 기록 길이: 0

분기 결정: manse
→ 사주풀이/운세 해석 관련 질문으로 판단되어 'manse' 에이

In [ ]:


# General QA 에이전트 테스트 케이스
general_qa_test_cases = [
    "오늘 뭐 먹지?",
    "기분전환 메뉴 추천",
    "공부법 알려줘",
    "영어회화 공부법",
    "다이어트에 좋은 음식이 뭔가요?",
    "미국 증시 전망 어떻게 보세요?",
    "오늘 날씨 어때요?",
    "스트레스 해소 방법 알려줘",
    "운동 추천해주세요"
]

print("=== General QA 에이전트 테스트 ===")
for i, question in enumerate(general_qa_test_cases, 1):
    print(f"\n{i}. ")
    test_supervisor_routing(question)
    print("-" * 50)

# 경계 케이스 테스트
edge_cases = [
    "사주에 대해 알려줘",  # 개념 vs 해석 애매
    "오행과 건강의 관계",  # 개념 + 일반 지식
    "사주로 보는 건강법",  # 사주 + 일반 지식
    "명리학 공부법",  # 개념 + 공부법
    "사주 용어 정리",  # 개념 정리
    "운세 보는 방법",  # 방법론
    "사주와 심리학",  # 학제간
    "오행 음식 추천",  # 사주 + 일반 추천
    "사주 관련 책 추천"  # 추천
]

print("=== 경계 케이스 테스트 ===")
for i, question in enumerate(edge_cases, 1):
    print(f"\n{i}. ")
    test_supervisor_routing(question)
    print("-" * 50)

# 대화 맥락 테스트
print("=== 대화 맥락 테스트 ===")

# 시나리오 1: 일반 대화 후 사주 질문
print("\n시나리오 1: 일반 대화 → 사주 질문")
chat_history_1 = [
    HumanMessage(content="오늘 날씨 어때요?"),
    AIMessage(content="오늘은 맑고 따뜻합니다.")
]
test_supervisor_routing("1995년생 남자 운세 봐주세요", chat_history_1)

print("\n" + "="*60)

# 시나리오 2: 사주 개념 질문 후 구체적 운세
print("\n시나리오 2: 사주 개념 → 구체적 운세")
chat_history_2 = [
    HumanMessage(content="겁재가 뭐야?"),
    AIMessage(content="겁재는 십신 중 하나로...")
]
test_supervisor_routing("내 겁재는 어떨까요?", chat_history_2)

print("\n" + "="*60)

# 시나리오 3: 일반 질문 연속
print("\n시나리오 3: 일반 질문 연속")
chat_history_3 = [
    HumanMessage(content="다이어트 음식 추천해주세요"),
    AIMessage(content="단백질이 풍부한 음식을 추천합니다...")
]
test_supervisor_routing("운동도 추천해주세요", chat_history_3)

# 프롬프트 개선 제안 출력
print("=== 프롬프트 개선 제안 ===")
print("""
1. 경계 케이스 처리 개선:
   - '사주에 대해 알려줘' 같은 애매한 질문에 대한 명확한 기준 추가
   - '사주로 보는 ~' 형태의 질문에 대한 분기 기준 명확화

2. 키워드 기반 분기 강화:
   - 각 에이전트별 핵심 키워드 목록 추가
   - 키워드 우선순위 설정

3. 맥락 인식 개선:
   - 이전 대화 내용을 고려한 분기 로직 강화
   - 연속 질문에 대한 처리 기준 명확화

4. 예외 처리 추가:
   - 분류가 어려운 질문에 대한 기본값 설정
   - 오류 발생 시 대체 분기 로직
""")
